In [1]:
import re

def read_file(file_path):
    with open(file_path, "r") as f:
        return f.read()

def parse_entities_v3(data):
    entities = {}
    entity_data = {}
    entity_sections = re.split(r"--- (\w+) ---", data)
    
    for i in range(1, len(entity_sections), 2):
        entity_name = entity_sections[i]
        attributes = entity_sections[i+1].strip().split("\n")
        for attribute in attributes:
            if "=" not in attribute:
                continue
            key, value = attribute.split("=")
            key = key.strip('"')
            if '"' in value:
                value = value.strip('"')
            elif '.' in value:
                value = float(value)
            elif value.isdigit():
                value = int(value)
            entity_data[key] = value
        if entity_name not in entities:
            entities[entity_name] = []
        entities[entity_name].append(entity_data.copy())
        entity_data.clear()
        
    return entities

parsed_entities_v3 = parse_entities_v3(read_file("generated_entities.txt"))
parsed_entities_v3

{'GuildName': [{'id': 126,
   'name': 'Gold Brokers',
   'description': 'PvP',
   'leader': 'Raelan',
   'members': 694,
   'founded_year': 485},
  {'id': 303,
   'name': 'The Silent Footsteps',
   'description': 'Knowledge',
   'leader': 'Thorgal',
   'members': 776,
   'founded_year': 761},
  {'id': 29,
   'name': 'Circle of Druids',
   'description': 'Questing',
   'leader': 'Thera',
   'members': 445,
   'founded_year': 643},
  {'id': 146,
   'name': 'Knights of Valor',
   'description': 'Spying',
   'leader': 'Zephyrix',
   'members': 122,
   'founded_year': 304},
  {'id': 285,
   'name': 'Mind Over Matter',
   'description': 'Trading',
   'leader': 'Morwyn',
   'members': 561,
   'founded_year': 870},
  {'id': 193,
   'name': 'Abyss Watchers',
   'description': 'PvP',
   'leader': 'Zephyrix',
   'members': 760,
   'founded_year': 966},
  {'id': 267,
   'name': 'Storm Riders',
   'description': 'Exploration',
   'leader': 'Valthor',
   'members': 457,
   'founded_year': 166},
  {'

In [16]:
import pymysql

def insert_into_mysql(entities, host, user, password, db_name):
    
    table_mapping = {
        "Character": "user_character"
    }
    
    connection = pymysql.connect(host=host,
                                 user=user,
                                 password=password,
                                 db=db_name,
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    
    try:
        with connection.cursor() as cursor:
            for entity, records in entities.items():
                table_name = table_mapping.get(entity, entity)
                for record in records:
                    columns = ', '.join(record.keys())
                    placeholders = ', '.join(['%s'] * len(record))
                    
                    sql = f"INSERT IGNORE INTO `{table_name}` ({columns}) VALUES ({placeholders})"
                    cursor.execute(sql, list(record.values()))
        
        connection.commit()

    finally:
        connection.close()
        
insert_into_mysql(parsed_entities_v3, 'localhost', 'root', 'password', 'db5')
